<a href="https://colab.research.google.com/github/lucapa17/Machine-Learning-Project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset

https://www.kaggle.com/datasets/kyr7plus/emg-4


In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from random import sample
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier



In [ ]:
folder = 'drive/MyDrive/project_ML/'
nl = 500
nt = 200
for i in range(0,4):
  file = folder + '%d.csv' %i
  D = pd.read_csv(file).to_numpy()
  D = np.delete(D, 64, 1)     
  j = sample(range(D.shape[0]),nl+nt)
  D = D[j,:]
  if (i == 0):
    XL = D[range(0,nl),:]
    YL = i*np.ones(nl) 
    XT = D[range(nl,nl+nt),:]
    YT = i*np.ones(nt)
  else:
    XL = np.concatenate((XL,D[range(0,nl),:]),axis=0)
    YL = np.append(YL,i*np.ones(nl))
    XT = np.concatenate((XT,D[range(nl,nl+nt),:]),axis=0)
    YT = np.append(YT,i*np.ones(nt))


In [ ]:
# Normalization of the data
scalerX = preprocessing.MinMaxScaler()
XL = scalerX.fit_transform(XL)
XT = scalerX.transform(XT)

In [ ]:
# KRR
from sklearn.kernel_ridge import KernelRidge

param_grid = {'estimator__kernel': ['linear', 'poly', 'rbf'],
              'estimator__alpha': np.logspace(-4,3,5),
              'estimator__gamma': np.logspace(-4,3,5)}

grid_search = GridSearchCV(OneVsRestClassifier(KernelRidge()),
                           param_grid,
                           scoring = 'balanced_accuracy',
                           cv=10,
                           verbose=0)
grid_search.fit(XL, YL)

print("Best model:", grid_search.best_estimator_)

YP = grid_search.predict(XT)

accuracy = np.mean(YP == YT)
print(accuracy)
confusion_matrix(YT,YP)

In [ ]:
# SVC

from sklearn.svm import SVC

param_grid = {'kernel': ['linear', 'poly', 'rbf'],
              'C': np.logspace(-4,3,5),
              'gamma': np.logspace(-4,3,5)}

grid_search = GridSearchCV(estimator = SVC(),
                          param_grid = param_grid,
                          scoring = 'balanced_accuracy',
                          cv = 10,
                          verbose = 0)
grid_search.fit(XL,YL)

print("Best model:", grid_search.best_params_)

YP = grid_search.predict(XT)

accuracy = np.mean(YP == YT)
print(accuracy)
confusion_matrix(YT,YP)

In [ ]:
# RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier

M = RandomForestClassifier(n_estimators = 1000)
M.fit(XL,YL)
YP = M.predict(XT)

accuracy = np.mean(YP == YT)
print(accuracy)
confusion_matrix(YT,YP)